In [1]:
from importlib import reload
import sys

src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

max_length = 128

train = True
debias = False

In [2]:
import data_generator;reload(data_generator)

train_data, val_data, test_data = data_generator.GetData(max_length, sample=1000)

INFO:absl:Using /tmp/tfhub_modules to cache modules.
100%|██████████| 3000/3000 [00:00<00:00, 3381.72it/s]


        tag  cat  occurences
0    B-MISC    0           6
1     I-LOC    1        2526
2    I-MISC    2        1017
3     I-ORG    3        1866
4     I-PER    4        8414
5         O    5       52303
6  [nerCLS]    6        3000
7  [nerPAD]    7      295298
8  [nerSEP]    8        3000
9    [nerX]    9       16570

                tag  cat  occurences
0  AFRICAN-AMERICAN    0        1527
1          EUROPEAN    1        1473
2         [raceCLS]    2        3000
3         [racePAD]    3      295298
4         [raceSEP]    4        3000
5           [raceX]    5       79702

           tag  cat  occurences
0       FEMALE    0        1436
1         MALE    1        1564
2  [genderCLS]    2        3000
3  [genderPAD]    3      295298
4  [genderSEP]    4        3000
5    [genderX]    5       79702



In [3]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
import model_utils; reload(model_utils)
    
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = model_utils.NER(max_length)
    
model.generate(bert_train_layers=4)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, None, 768)    108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [4]:
if train:

    model.fit(
        sess,
        train_data,
        val_data,
        epochs=8,
        batch_size=32,
        debias=debias
    ) 
    
else: 
    model_utils.initialize_vars(sess)


val_acc_ner: 0.86; val_acc_ner_non_other: 0.32;  val_acc_gender: 0.30; val_acc_race: 0.53



val_acc_ner: 0.94; val_acc_ner_non_other: 0.72;  val_acc_gender: 0.50; val_acc_race: 0.52



val_acc_ner: 0.96; val_acc_ner_non_other: 0.82;  val_acc_gender: 0.51; val_acc_race: 0.52



val_acc_ner: 0.97; val_acc_ner_non_other: 0.89;  val_acc_gender: 0.50; val_acc_race: 0.52



val_acc_ner: 0.98; val_acc_ner_non_other: 0.90;  val_acc_gender: 0.50; val_acc_race: 0.49



val_acc_ner: 0.98; val_acc_ner_non_other: 0.92;  val_acc_gender: 0.49; val_acc_race: 0.48



val_acc_ner: 0.98; val_acc_ner_non_other: 0.93;  val_acc_gender: 0.49; val_acc_race: 0.48



val_acc_ner: 0.98; val_acc_ner_non_other: 0.93;  val_acc_gender: 0.50; val_acc_race: 0.48



In [ ]:
bias = model.getBiasedPValues(test_data, num_iterations=1000)

In [ ]:
bias

In [ ]:
sess.close()